In [1]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
#Json file option
filejson = "C:/Users/danie/Desktop/bbc_news_list_uk.json"
filecsv = "C:/Users/danie/Desktop/bbc_news_list_uk.csv"
article_df = pd.read_csv(filecsv)
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15825 entries, 0 to 15824
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              15825 non-null  int64 
 1   tags               1 non-null      object
 2   title              15825 non-null  object
 3   news_post_date     15825 non-null  object
 4   raw_content        15468 non-null  object
 5   content            15468 non-null  object
 6   url                15825 non-null  object
 7   author             1453 non-null   object
 8   language           15825 non-null  object
 9   id                 15825 non-null  object
 10  region             15488 non-null  object
 11  short_description  15825 non-null  object
 12  category           15825 non-null  object
 13  crawled_at         15825 non-null  object
 14  Article_Number     15825 non-null  int32 
dtypes: int32(1), int64(1), object(13)
memory usage: 1.8+ MB


In [16]:
#filename = 'articles.pkl'

#article_df = pd.read_pickle(filename)
#article_df = article_df.assign(Article_Number=range(len(article_df)))
#article_df = article_df.reset_index()
#article_df.info()

In [60]:
# tokenize sentences in an article 
import re

def split_sentences(article, article_id):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id in article_df[['content','Article_Number']].values:
    sentences = split_sentences(str(article), article_id)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id'])

In [62]:
def pronoun_occurances(text):
    """ This function will count the number of female and male pronoun occurences in a given sentence. 
    We will need to update the REGEX in order to incorporate more pronouns if we need to"""
    pattern_m = r'(\s|^)(he|his|him|he\'s|hes)\b' #this regex will capture he/his as standalone words within a string but also at beginning of sentence
    matches_m = re.findall(pattern_m, text, re.IGNORECASE) #IGNORECASE is necessary to make sure that it picks up the pronouns at the beginning of a sentence
    pattern_f = r'(\s|^)(she|her|hers|shes|she\'s)\b'
    matches_f = re.findall(pattern_f, text, re.IGNORECASE)
    count_m = len(matches_m)
    count_f = len(matches_f)
    return count_f, count_m

In [63]:
#create a variable applying the function of pronoun occurence
sent = sentences_df['sentences'].apply(pronoun_occurances)
# Create two new columns in sentences DF from the tuple output in "sent"
sentences_df['female_count'] = [x[0] for x in sent]
sentences_df['male_count']= [x[1] for x in sent]

#Bug is fixed and now it counts properly
sentences_df

,sentences,article_id,female_count,male_count
0,The heroin substitute methadone can be used as a way of weaning addicts off heroin but the substitute can also become addictive.,0,0,0
1,"Earlier this year a debate broke out in Scotland when Professor Neil McKeganey, director of the Centre for Drug Misuse Research at the University of Glasgow, said more effort was needed to get people off drugs, including methadone, through abstinence.",0,0,0
2,"But a group of 40 specialists, including university professors and doctors who treat addicts, responded: ""If policy makers were to heed the critics' advice to close down methadone treatment or impose an arbitrary time limit on its administration, the community can anticipate more overdose deaths, more HIV and more crime.""",0,0,0
3,So what do recovering addicts think?,0,0,0
4,"Chris used methadone for five years to help wean him off heroin but felt he had to quit the drug substitute last October""I just thought, I've got to get a grip here, because I've been in and out of prison since I was 16 - that's half my life,"" he told The Report.",0,0,3
...,...,...,...,...
225660,"Similar moves in Europe have sparked cries of discrimination against Muslims, reports the Guardian.",15824,0,0
225661,The lead in the Daily Mail is a claim that councils are secretly analysing the contents of people's dustbins.,15824,0,0
225662,"It says authorities are using the information gathered to place households into social categories, like ""wealthy achievers"" and ""hard-pressed"", in order to target them in future leafleting campaigns.",15824,0,0
225663,The Daily Express leads with a warning from US scientists that some household cleaning agents can increase the likelihood of developing breast cancer.,15824,0,0


In [66]:
sentences_df.sort_values(by='male_count')

,sentences,article_id,female_count,male_count
0,The heroin substitute methadone can be used as a way of weaning addicts off heroin but the substitute can also become addictive.,0,0,0
143203,"The Parades Commission ruled that marchers should not go past the house of Kevin McDaid, 49, who was beaten to death last year.",9808,0,0
143204,The Catholic father of four was killed in the Somerset Drive area in May 2009 and Damian Fleming was badly injured.,9808,0,0
143205,"Robert McIlreavy from Killowen Orange Lodge said it was ""disappointing"".",9808,0,0
143207,"""We thought we had done enough last year to lower tensions.""",9808,0,0
...,...,...,...,...
117658,"Wallace ran from the party and when he saw police at his mother's home, he ended up in a house in Newtownards where he told a friend that Ms Hughes had been ""winding him up and making a fool of him in front of his mates"".",7970,0,7
81524,"The lawyer said Wallace fled from the party and when he saw police at his mother's home, he ended up in a house in Newtownards where he told a friend that Ms Hughes had been ""winding him up and making a fool of him in front of his mates"".",5432,0,7
202203,"Qassir Tamuri, 19, of east London, had his 30-month term cut to 12 monthsAsim Alhaddad, 24, of north-west London, had his two-year sentence reduced to 21 monthsMustafa Hassan, 19, of west London, had his term cut from 18 months to 10 monthsMohammed El-Araj, 20, of west London, had his two-year term replaced with an 18-month sentenceSid-Ali Zenaf, 20, of east London, had his two-year sentence cut to 18 monthsMosab Al-Ani, 22, of West Yorkshire, had his 12-month sentence quashed and replaced with a community order of 12 monthsMahamoud Farhan Ali, 19, west London, had his 18 month-sentence reduced to 10 monthsAndris Feodorovs, 30, of east London, Abdul Samad, 23, of west London, and Sayed Rizvi, 19, of south-west London, failed in their appeal bids",14006,0,7
105144,"He says he and his sister are close, and she paid for his eyebrow piercing, telling him ""it wouldn't hurt"" - so it was only natural he asked her to be his mayoress.",7077,2,7


In [67]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if male_col > female_col: 
        return "1"
    elif male_col == female_col: 
        return "0.5"
    else:
        return "0"

sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']), axis=1)
sentences_df



,sentences,article_id,female_count,male_count,col_type
0,The heroin substitute methadone can be used as a way of weaning addicts off heroin but the substitute can also become addictive.,0,0,0,0.5
1,"Earlier this year a debate broke out in Scotland when Professor Neil McKeganey, director of the Centre for Drug Misuse Research at the University of Glasgow, said more effort was needed to get people off drugs, including methadone, through abstinence.",0,0,0,0.5
2,"But a group of 40 specialists, including university professors and doctors who treat addicts, responded: ""If policy makers were to heed the critics' advice to close down methadone treatment or impose an arbitrary time limit on its administration, the community can anticipate more overdose deaths, more HIV and more crime.""",0,0,0,0.5
3,So what do recovering addicts think?,0,0,0,0.5
4,"Chris used methadone for five years to help wean him off heroin but felt he had to quit the drug substitute last October""I just thought, I've got to get a grip here, because I've been in and out of prison since I was 16 - that's half my life,"" he told The Report.",0,0,3,1
...,...,...,...,...,...
225660,"Similar moves in Europe have sparked cries of discrimination against Muslims, reports the Guardian.",15824,0,0,0.5
225661,The lead in the Daily Mail is a claim that councils are secretly analysing the contents of people's dustbins.,15824,0,0,0.5
225662,"It says authorities are using the information gathered to place households into social categories, like ""wealthy achievers"" and ""hard-pressed"", in order to target them in future leafleting campaigns.",15824,0,0,0.5
225663,The Daily Express leads with a warning from US scientists that some household cleaning agents can increase the likelihood of developing breast cancer.,15824,0,0,0.5


**Building the Classifier**

**Logistics Regression Classifier**

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.corpus import stopwords

In [72]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [76]:
#Sentence Encoding

sentences_df['encoded_sentences'] = sentences_df.loc[:,'sentences']

def tidy_text(sentence, remove_stopwords = True):
   
    # Convert words to lower case
    sentence = sentence.lower()
    
    # Expand contractions
    if True:
        sentence = sentence.split()
        new_text = []
        for word in sentence:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        sentence = " ".join(new_text)
    
    # Format words and remove unwanted characters
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r'\'', ' ', sentence)
    
    # remove stopwords
    if remove_stopwords:
        sentence = sentence.split()
        stops = set(stopwords.words("english"))
        sentence = [w for w in sentence if not w in stops]
        sentence = " ".join(sentence)

    # Tokenize each word
    sentence =  nltk.WordPunctTokenizer().tokenize(sentence)
    
    # Lemmatize each token
    lemm = nltk.stem.WordNetLemmatizer()
    sentence = list(map(lambda word:list(map(lemm.lemmatize, word)), sentence))
    
    return sentence

sentences_df['encoded_sentences'] = sentences_df['encoded_sentences'].apply(tidy_text)

In [77]:
sentences_df

,sentences,article_id,female_count,male_count,col_type,encoded_sentences
0,The heroin substitute methadone can be used as a way of weaning addicts off heroin but the substitute can also become addictive.,0,0,0,0.5,"[[h, e, r, o, i, n], [s, u, b, s, t, i, t, u, t, e], [m, e, t, h, a, d, o, n, e], [u, s, e, d], [w, a, y], [w, e, a, n, i, n, g], [a, d, d, i, c, t, s], [h, e, r, o, i, n], [s, u, b, s, t, i, t, u, t, e], [a, l, s, o], [b, e, c, o, m, e], [a, d, d, i, c, t, i, v, e]]"
1,"Earlier this year a debate broke out in Scotland when Professor Neil McKeganey, director of the Centre for Drug Misuse Research at the University of Glasgow, said more effort was needed to get people off drugs, including methadone, through abstinence.",0,0,0,0.5,"[[e, a, r, l, i, e, r], [y, e, a, r], [d, e, b, a, t, e], [b, r, o, k, e], [s, c, o, t, l, a, n, d], [p, r, o, f, e, s, s, o, r], [n, e, i, l], [m, c, k, e, g, a, n, e, y], [d, i, r, e, c, t, o, r], [c, e, n, t, r, e], [d, r, u, g], [m, i, s, u, s, e], [r, e, s, e, a, r, c, h], [u, n, i, v, e, r, s, i, t, y], [g, l, a, s, g, o, w], [s, a, i, d], [e, f, f, o, r, t], [n, e, e, d, e, d], [g, e, t], [p, e, o, p, l, e], [d, r, u, g, s], [i, n, c, l, u, d, i, n, g], [m, e, t, h, a, d, o, n, e], [a, b, s, t, i, n, e, n, c, e]]"
2,"But a group of 40 specialists, including university professors and doctors who treat addicts, responded: ""If policy makers were to heed the critics' advice to close down methadone treatment or impose an arbitrary time limit on its administration, the community can anticipate more overdose deaths, more HIV and more crime.""",0,0,0,0.5,"[[g, r, o, u, p], [4, 0], [s, p, e, c, i, a, l, i, s, t, s], [i, n, c, l, u, d, i, n, g], [u, n, i, v, e, r, s, i, t, y], [p, r, o, f, e, s, s, o, r, s], [d, o, c, t, o, r, s], [t, r, e, a, t], [a, d, d, i, c, t, s], [r, e, s, p, o, n, d, e, d], [p, o, l, i, c, y], [m, a, k, e, r, s], [h, e, e, d], [c, r, i, t, i, c, s], [a, d, v, i, c, e], [c, l, o, s, e], [m, e, t, h, a, d, o, n, e], [t, r, e, a, t, m, e, n, t], [i, m, p, o, s, e], [a, r, b, i, t, r, a, r, y], [t, i, m, e], [l, i, m, i, t], [a, d, m, i, n, i, s, t, r, a, t, i, o, n], [c, o, m, m, u, n, i, t, y], [a, n, t, i, c, i, p, a, t, e], [o, v, e, r, d, o, s, e], [d, e, a, t, h, s], [h, i, v], [c, r, i, m, e]]"
3,So what do recovering addicts think?,0,0,0,0.5,"[[r, e, c, o, v, e, r, i, n, g], [a, d, d, i, c, t, s], [t, h, i, n, k]]"
4,"Chris used methadone for five years to help wean him off heroin but felt he had to quit the drug substitute last October""I just thought, I've got to get a grip here, because I've been in and out of prison since I was 16 - that's half my life,"" he told The Report.",0,0,3,1,"[[c, h, r, i, s], [u, s, e, d], [m, e, t, h, a, d, o, n, e], [f, i, v, e], [y, e, a, r, s], [h, e, l, p], [w, e, a, n], [h, e, r, o, i, n], [f, e, l, t], [q, u, i, t], [d, r, u, g], [s, u, b, s, t, i, t, u, t, e], [l, a, s, t], [o, c, t, o, b, e, r], [t, h, o, u, g, h, t], [g, o, t], [g, e, t], [g, r, i, p], [p, r, i, s, o, n], [s, i, n, c, e], [1, 6], [h, a, l, f], [l, i, f, e], [t, o, l, d], [r, e, p, o, r, t]]"
...,...,...,...,...,...,...
225660,"Similar moves in Europe have sparked cries of discrimination against Muslims, reports the Guardian.",15824,0,0,0.5,"[[s, i, m, i, l, a, r], [m, o, v, e, s], [e, u, r, o, p, e], [s, p, a, r, k, e, d], [c, r, i, e, s], [d, i, s, c, r, i, m, i, n, a, t, i, o, n], [m, u, s, l, i, m, s], [r, e, p, o, r, t, s], [g, u, a, r, d, i, a, n]]"
225661,The lead in the Daily Mail is a claim that councils are secretly analysing the contents of people's dustbins.,15824,0,0,0.5,"[[l, e, a, d], [d, a, i, l, y], [m, a, i, l], [c, l, a, i, m], [c, o, u, n, c, i, l, s], [s, e, c, r, e, t, l, y], [a, n, a, l, y, s, i, n, g], [c, o, n, t, e, n, t, s], [p, e, o, p, l, e], [d, u, s, t, b, i, n, s]]"
225662,"It says authorities are using the information gathered to place households into social categories, like ""wealthy achievers"" and ""hard-pressed"", in orde

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_converter = CountVectorizer(tokenizer=lambda doc: doc)

#this should be changed to only the male and female sents. 
x = bow_converter.fit_transform(sentence_df['encoded_sentences'])
words = bow_converter.get_feature_names()
len(words)

In [ ]:
#splitting the data into training and test set 
X = sentences_df[["female_count", "male_count"]] #input 
y = sentences_df["col_type"] #label 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

#Note: using sentences_df["sentences"] creates an error 

# I think we are getting an error here because we haven't encoded the sentences. I'm not sure if I should work on this or the data set. I'm thinking BOW 
#and TFIDF for what we should use to encode. Also I do think we need to clean up the sentences. ie punctuation, lowercase, contractions, tokenazion, 
# and maybe stemming? 


In [ ]:
#multinomial logistic regression 
logreg = LogisticRegression(multi_class = "multinomial", solver = "lbfgs") #classifier 
logreg.fit(X_train, y_train) #fit the model 
logreg.score(X_train, y_train) #get accuracy

In [ ]:
logreg.coef_

In [ ]:
pd.DataFrame(zip(X_train.columns, np.transpose(abs(logreg.coef_))), columns=['features', 'coef']) #use absolute values to identify biggest coeffs

Interestingly, male coefficient for neutral is higher than the female. Otherwise, as expected, the coefficients for female and male each are correspondingly high for each gender. 

In [ ]:
prediction = logreg.predict(X_test) #predict test 
print(metrics.accuracy_score(y_test, prediction)) #accuracy 
print(metrics.confusion_matrix(y_test, prediction)) #confusion matrix
print(metrics.classification_report(y_test, prediction))

In [ ]:
#I think with the above solution we wouldn't need this function 
def count_words(text, word_list):
    return sum(text.count(word) for word in word_list)

sentences_df['male_count2'] = sentences_df['sentences'].apply(count_words, word_list=his_w)
sentences_df['female_count2'] = sentences_df['sentences'].apply(count_words, word_list=her_w)


In [ ]:
pd.set_option('display.max_colwidth', 1000)
sentences_df.loc[[7]]

#there is a bug here. It weirdly seems to be double counting? The zip function is new to me though so maybe thats the problem. 

In [ ]:
df = pd.DataFrame({'text': ['This is a sample text', 'Another text example', 'One more example']})

# define two lists of specific words to count
word_list1 = ['text', 'example']
word_list2 = ['is', 'more']

def count_words(text, word_list):
    return sum(text.count(word) for word in word_list)

# use apply() to add two new columns with the counts of the specific words in each list
df['word_count1'] = df['text'].apply(count_words, word_list=word_list1)
df['word_count2'] = df['text'].apply(count_words, word_list=word_list2)

# print the resulting DataFrame
print(df)

